In [1]:
import aocd
from intcode import Intcode
from collections import deque

In [2]:
instructions = [int(n) for n in aocd.get_data(day=23, year=2019).split(',')]

In [3]:
class Computer:
    address: int
    controller: Intcode
    input_queue: deque
    output_queue: deque
    missed_inputs: int

    def __init__(self, address):
        self.address = address
        self.input_queue = deque([address])
        self.output_queue = deque()
        self.controller = Intcode(instructions, self.read_input, self.output_queue)
        self.missed_inputs = 0
    
    @property
    def packet(self):
        if len(self.output_queue) < 3:
            return None
        address = self.output_queue.popleft()
        x = self.output_queue.popleft()
        y = self.output_queue.popleft()
        return address, x, y
    
    def recieve(self, packet):
        _, x, y = packet
        self.input_queue.extend((x, y))
    
    def read_input(self):
        if not self.input_queue:
            self.missed_inputs += 1
            return -1
        self.missed_inputs = 0
        return self.input_queue.popleft()
    
    @property
    def is_idle(self):
        return self.missed_inputs > 1 and not self.input_queue and not self.output_queue

In [4]:
class NAT:
    computers: list[Computer]
    last_packet: tuple[int, int, int] | None = None
    log: list[int]
    looped = False

    def __init__(self, computers):
        self.computers = computers
        self.log = []

    def recieve(self, packet):
        if not self.last_packet:
            print("Part 1:", packet[2])
        self.last_packet = packet
    
    def monitor(self):
        if all(c.is_idle for c in self.computers):
            assert self.last_packet
            self.computers[0].recieve(self.last_packet)
            self.log.append(self.last_packet[2])
            if len(self.log) >= 2 and self.log[-1] == self.log[-2]:
                self.looped = True
                print("Part 2:", self.log[-1])
                

In [5]:
class Network:
    computers: list[Computer]
    nat: NAT

    def __init__(self, num_computers):
        self.computers = [Computer(address) for address in range(num_computers)]
        self.nat = NAT(self.computers)
    
    def send(self, packet):
        address = packet[0]
        if address == 255:
            self.nat.recieve(packet)
        else:
            self.computers[address].recieve(packet)
    
    def run(self):
        while not self.nat.looped:
            for computer in self.computers:
                computer.controller.fetch_exec()
                if packet := computer.packet:
                    self.send(packet)
            self.nat.monitor()

In [6]:
Network(num_computers=50).run()

Part 1: 17849
Part 2: 12235
